# Administrative Descriptive Stats - Merge results

This notebook simply merges all the separate tabular results determined in the previous notebook.  The outputs are the final tabular results with average service access times for different season-mode combinations, at specified adm levels.

In [ ]:
import os, sys
os.environ['USE_PYGEOS'] = '0'
from datetime import date

import pandas as pd
import geopandas as gpd
import numpy as np
from scipy import stats

import re

import dask.dataframe as dd

import json
import csv

In [ ]:
data_root = 'D:\\github_test\\'

##################################################################
##################################################################
#read project input parameters that will eventually be passed from the UI
data_file = data_root + 'project_data.json'

##################################################################
##################################################################
#read project variables that will come from UI so that we have our parameters and file locations
with open(data_file, 'rb') as f:
    data_loaded = json.load(f)
f.close()

##################################################################
##################################################################
#read information from the project setup file that's relevant to this section of code
#imports
local_population_folder = data_loaded['local_population_folder']
local_lc_folder = data_loaded['local_lc_folder']
access_dir = data_loaded['access_dir']
dest_crs = data_loaded['dest_crs']
dest_crs_id = data_loaded['dest_crs_id']

local_boundaries_folder = data_loaded['local_boundaries_folder']
level = data_loaded['level']
max_level = data_loaded['max_level']

if level != 'custom':
    shapefile_adm_field = data_loaded['shapefile_adm_field']
    adm_name = data_loaded['adm_name']

seasons = sorted([os.path.join(local_lc_folder,file) \
            for file \
            in os.listdir(local_lc_folder) \
            if file.endswith(".tif")])

for strnum in range(0, len(seasons)):
    seasons[strnum] = str.replace(seasons[strnum], local_lc_folder,"")
    seasons[strnum] = str.replace(seasons[strnum], ".tif","")   

In [ ]:
mode_list =['walk','multi']
ssn_mode_list = []
for ssn in seasons:
    for mode_num in range(0,len(mode_list)):
        ssn_mode_list.append(ssn + "_" + mode_list[mode_num])


In [ ]:
#read a csv file to get the 'base' adm columns
folder_path = access_dir + 'tables' + '\\separate\\'
for filename in os.listdir(folder_path):
    # Check if the file is a CSV file
    if filename.endswith('adm2_final.csv'):
        # Construct the full path to the CSV file
        file_path_adm2 = os.path.join(folder_path, filename)
    if filename.endswith('adm3_final.csv'):
        # Construct the full path to the CSV file
        file_path_adm3 = os.path.join(folder_path, filename)

dummy_URL = file_path_adm2
Adm2_Code = pd.read_csv(os.path.join(dummy_URL),header=0,
                                                usecols = [0,1])\
                                                .sort_values('Adm2_Code')
adm2_base = Adm2_Code.copy()

if max_level == 'adm3':
    dummy_URL = file_path_adm3
    Adm3_Code = pd.read_csv(os.path.join(dummy_URL),header=0,
                                                    usecols = [0,1,2,3])\
                                                    .sort_values('Adm3_Code') 
    adm3_base = Adm3_Code.copy()

In [ ]:
adm2_final = adm2_base
if max_level == 'adm3':
    adm3_final = adm3_base

adm2_start = 1
adm3_start = 1

for ssn_mode_index in range(0,len(ssn_mode_list)):
# for ssn_mode_index in range(0,1):
    ssn_mode = ssn_mode_list[ssn_mode_index]
    print(ssn_mode)
        
    df_URL = access_dir + 'pixels' + '\\' + f"{ssn_mode}_df_pixels_final-*.csv"
    df_pixels_source = dd.read_csv(os.path.join(df_URL),header=0,
                                                na_values = ' ',
                                                # usecols = col_range,
                                                blocksize=100e6).head(n=1)

    # Get the list of column names
    column_names = df_pixels_source.columns

    # List to store the column indices
    column_indices = []

    # Loop through column names to find the ones ending with "_avg_adm2" or "_avg_adm3"
    for idx, col_name in enumerate(column_names):
        if col_name.endswith('_avg_adm2') or col_name.endswith('_avg_adm3'):
            column_indices.append(idx)
                 
    min_col_num = column_indices[0]
    max_col_num = column_indices[len(column_indices)-1]+1  
    
    col_range = range(min_col_num,max_col_num)
    service_index_range = range(0,max_col_num-min_col_num)

    df_pixels = df_pixels_source.iloc[: ,col_range].copy()    
    del df_pixels_source
    col_list = df_pixels.columns
 
    if adm2_start == 1:
        adm2_first = 1
        adm2_start = 0
    else:
        adm2_first = 0

    if adm3_start == 1:
        adm3_first = 1
        adm3_start = 0
    else:
        adm3_first = 0
    
    for service_index in service_index_range:
        read_service = col_list[service_index]
        read_service_adm = read_service
        read_service = read_service_adm[:-9]
        adm_level = read_service_adm[-1]

        # first we merge average access times
        ##############
        if adm_level == '2':            
        
            csv_URL = access_dir + 'tables' + '\\separate\\' + f"{read_service_adm}_final.csv"
            current_data = pd.read_csv(os.path.join(csv_URL),header=0)\
                                                .sort_values('Adm2_Code')

            column = current_data[["Adm2_Code",read_service_adm,read_service_adm+'_idx']].copy()
            adm2_final = pd.merge(adm2_final,column,on=["Adm2_Code"])

        else:

            csv_URL = access_dir + 'tables' + '\\separate\\' + f"{read_service_adm}_final.csv"
            current_data = pd.read_csv(os.path.join(csv_URL),header=0)\
                                                .sort_values('Adm3_Code')
    
            column = current_data[["Adm3_Code",read_service_adm,read_service_adm+'_idx']].copy()
            adm3_final = pd.merge(adm3_final,column,on=["Adm3_Code"])

        # Now we merge travel time bins
        ############        
        if adm_level == '2':
            csv_URL = access_dir + 'tables' + '\\separate\\' + f"{read_service_adm}_acc_indicators_long.csv"
            current_long_data_adm2 = pd.read_csv(os.path.join(csv_URL),header=0,\
                                                usecols = [0,1,2,3,4,5,6,7])

            if adm2_first == 1:
                adm2_long_final = current_long_data_adm2
                adm2_first = 0
            else:
                adm2_long_final = pd.concat([adm2_long_final,current_long_data_adm2])

        else:
            csv_URL = access_dir + 'tables' + '\\separate\\' + f"{read_service_adm}_acc_indicators_long.csv"
            current_long_data_adm3 = pd.read_csv(os.path.join(csv_URL),header=0,\
                                                usecols = [0,1,2,3,4,5,6,7,8,9])
            if adm3_first == 1:
                adm3_long_final = current_long_data_adm3
                adm3_first = 0
            else:
                adm3_long_final = pd.concat([adm3_long_final,current_long_data_adm3])

# Identify columns ending with '_idx'
idx_columns = [col for col in adm2_final.columns if col.endswith('_idx')]
other_columns = [col for col in adm2_final.columns if not col.endswith('_idx')]
# Rearrange columns
adm2_final = adm2_final[other_columns + idx_columns]

# export final csv's!!!
adm2_final.to_csv(access_dir + "tables\\adm2_mean_final.csv",index=False)
if max_level == 'adm3':
    # Identify columns ending with '_idx'
    idx_columns = [col for col in adm3_final.columns if col.endswith('_idx')]
    other_columns = [col for col in adm3_final.columns if not col.endswith('_idx')]
    # Rearrange columns
    adm3_final = adm3_final[other_columns + idx_columns]
    adm3_final.to_csv(access_dir + "tables\\adm3_mean_final.csv",index=False)

adm2_long_final.to_csv(access_dir + "tables\\adm2_long_final.csv",index=False)
if max_level == 'adm3':
    adm3_long_final.to_csv(access_dir + "tables\\adm3_long_final.csv",index=False)